In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef,
    confusion_matrix, roc_auc_score, average_precision_score
)
import random
import os
from tqdm import tqdm

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [2]:
# 数据集类 - 使用平均池化
class ProteinNPYDataset(Dataset):
    def __init__(self, pos_path, neg_path):
        self.pos = np.load(pos_path, mmap_mode='r')
        self.neg = np.load(neg_path, mmap_mode='r')
        self.lengths = [len(self.pos), len(self.neg)]
        self.total_len = self.lengths[0] + self.lengths[1]

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        if idx < self.lengths[0]:
            x = self.pos[idx]  # (300, 1152)
            y = 1
        else:
            x = self.neg[idx - self.lengths[0]]  # (300, 1152)
            y = 0
        
        # 平均池化：将(300, 1152) -> (1152,)
        x_pooled = np.mean(x, axis=0)
        
        return torch.tensor(x_pooled, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

In [3]:
# GPU加速的基线模型
class LogisticRegressionGPU(nn.Module):
    def __init__(self, input_dim=1152, num_classes=2):
        super().__init__()
        self.linear = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        return self.linear(x)

class SVMGPU(nn.Module):
    def __init__(self, input_dim=1152, hidden_dim=512, num_classes=2):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim//2, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)

class RandomForestGPU(nn.Module):
    def __init__(self, input_dim=1152, hidden_dim=256, num_classes=2):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)

class GradientBoostingGPU(nn.Module):
    def __init__(self, input_dim=1152, hidden_dim=512, num_classes=2):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim//2, num_classes)
        )
    
    def forward(self, x):
        return self.classifier(x)

In [4]:
# 评估函数
def calculate_all_metrics(y_true, y_pred, y_proba):
    """计算所有评估指标"""
    # 基础指标
    acc = accuracy_score(y_true, y_pred)
    pre = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    # 混淆矩阵计算SN和SP
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sn = tp / (tp + fn) if (tp + fn) > 0 else 0  # 敏感性 (Sensitivity/Recall)
    sp = tn / (tn + fp) if (tn + fp) > 0 else 0  # 特异性 (Specificity)
    
    # AUC和AUPRC
    auc = roc_auc_score(y_true, y_proba)
    auprc = average_precision_score(y_true, y_proba)
    
    return {
        'ACC': acc, 'PRE': pre, 'REC': rec, 'F1': f1, 'MCC': mcc,
        'SN': sn, 'SP': sp, 'AUC': auc, 'AUPRC': auprc
    }

def print_metrics(metrics, prefix=""):
    """打印指标"""
    print(f"{prefix}ACC: {metrics['ACC']:.4f}, PRE: {metrics['PRE']:.4f}, REC: {metrics['REC']:.4f}, F1: {metrics['F1']:.4f}, MCC: {metrics['MCC']:.4f}")
    print(f"{prefix}SN: {metrics['SN']:.4f}, SP: {metrics['SP']:.4f}, AUC: {metrics['AUC']:.4f}, AUPRC: {metrics['AUPRC']:.4f}")

In [5]:
# 数据路径
train_pos = '/exp_data/sjx/star/first_data/ESM-embedding/positive_train_embedding.npy'
train_neg = '/exp_data/sjx/star/first_data/ESM-embedding/negative_train_embedding.npy'
test_pos = '/exp_data/sjx/star/first_data/ESM-embedding/positive_test_embedding.npy'
test_neg = '/exp_data/sjx/star/first_data/ESM-embedding/negative_test_embedding.npy'

# 构建全体索引和标签（不加载数据）
pos_len = np.load(train_pos, mmap_mode='r').shape[0]
neg_len = np.load(train_neg, mmap_mode='r').shape[0]
all_indices = np.concatenate([np.arange(pos_len), np.arange(neg_len) + pos_len])
all_labels = np.concatenate([np.ones(pos_len, dtype=int), np.zeros(neg_len, dtype=int)])

print(f"训练集正样本: {pos_len}, 负样本: {neg_len}")
print(f"总样本数: {len(all_indices)}")

# 数据集（只创建索引，不加载数据）
full_dataset = ProteinNPYDataset(train_pos, train_neg)
test_dataset = ProteinNPYDataset(test_pos, test_neg)

print(f"训练集大小: {len(full_dataset)}")
print(f"测试集大小: {len(test_dataset)}")
print(f"特征维度: 1152 (平均池化后)")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

训练集正样本: 3271, 负样本: 1323
总样本数: 4594
训练集大小: 4594
测试集大小: 1149
特征维度: 1152 (平均池化后)
使用设备: cuda


In [6]:
# K折分层
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 存储结果
models_results = {
    'LogisticRegression': {'cv': [], 'test': []},
    'SVM': {'cv': [], 'test': []},
    'RandomForest': {'cv': [], 'test': []},
    'GradientBoosting': {'cv': [], 'test': []}
}

# 记录最佳测试ACC
best_test_acc = {model: 0 for model in models_results.keys()}
best_test_fold = {model: 0 for model in models_results.keys()}
best_test_metrics = {model: None for model in models_results.keys()}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("开始十折交叉验证...")

开始十折交叉验证...


In [15]:
for fold, (train_idx, val_idx) in enumerate(skf.split(all_indices, all_labels), 1):
    print(f"\n{'='*60}")
    print(f"========== Fold {fold}/10 ==========")
    print(f"{'='*60}")
    
    print(f"训练集样本数: {len(train_idx)}, 验证集样本数: {len(val_idx)}")
    
    # 准备数据加载器（只传索引，不加载数据）
    train_loader = DataLoader(Subset(full_dataset, train_idx), batch_size=128, shuffle=True, num_workers=4)
    val_loader = DataLoader(Subset(full_dataset, val_idx), batch_size=128, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)
    
    # ========== 逻辑回归 ==========
    print(f"\n--- 逻辑回归 Fold {fold} ---")
    
    model = LogisticRegressionGPU().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    
    # 训练
    model.train()
    for epoch in range(20):
        for x, y in tqdm(train_loader, desc=f"逻辑回归训练 Epoch {epoch+1}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
    
    # 验证
    model.eval()
    val_preds, val_labels, val_probs = [], [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="逻辑回归验证"):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(y.cpu().numpy())
            val_probs.extend(probs[:, 1].cpu().numpy())
    
    val_metrics = calculate_all_metrics(val_labels, val_preds, val_probs)
    print_metrics(val_metrics, "验证集 ")
    models_results['LogisticRegression']['cv'].append(val_metrics)
    
    # 测试
    test_preds, test_labels, test_probs = [], [], []
    with torch.no_grad():
        for x, y in tqdm(test_loader, desc="逻辑回归测试"):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(y.cpu().numpy())
            test_probs.extend(probs[:, 1].cpu().numpy())
    
    test_metrics = calculate_all_metrics(test_labels, test_preds, test_probs)
    print_metrics(test_metrics, "测试集 ")
    models_results['LogisticRegression']['test'].append(test_metrics)
    
    if test_metrics['ACC'] > best_test_acc['LogisticRegression']:
        best_test_acc['LogisticRegression'] = test_metrics['ACC']
        best_test_fold['LogisticRegression'] = fold
        best_test_metrics['LogisticRegression'] = test_metrics
    
    # ========== SVM ==========
    print(f"\n--- SVM Fold {fold} ---")
    
    model = SVMGPU().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    
    # 训练
    model.train()
    for epoch in range(30):
        for x, y in tqdm(train_loader, desc=f"SVM训练 Epoch {epoch+1}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
    
    # 验证
    model.eval()
    val_preds, val_labels, val_probs = [], [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="SVM验证"):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(y.cpu().numpy())
            val_probs.extend(probs[:, 1].cpu().numpy())
    
    val_metrics = calculate_all_metrics(val_labels, val_preds, val_probs)
    print_metrics(val_metrics, "验证集 ")
    models_results['SVM']['cv'].append(val_metrics)
    
    # 测试
    test_preds, test_labels, test_probs = [], [], []
    with torch.no_grad():
        for x, y in tqdm(test_loader, desc="SVM测试"):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(y.cpu().numpy())
            test_probs.extend(probs[:, 1].cpu().numpy())
    
    test_metrics = calculate_all_metrics(test_labels, test_preds, test_probs)
    print_metrics(test_metrics, "测试集 ")
    models_results['SVM']['test'].append(test_metrics)
    
    if test_metrics['ACC'] > best_test_acc['SVM']:
        best_test_acc['SVM'] = test_metrics['ACC']
        best_test_fold['SVM'] = fold
        best_test_metrics['SVM'] = test_metrics
    
    # ========== 随机森林 ==========
    print(f"\n--- 随机森林 Fold {fold} ---")
    
    model = RandomForestGPU().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    
    # 训练
    model.train()
    for epoch in range(25):
        for x, y in tqdm(train_loader, desc=f"随机森林训练 Epoch {epoch+1}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
    
    # 验证
    model.eval()
    val_preds, val_labels, val_probs = [], [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="随机森林验证"):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(y.cpu().numpy())
            val_probs.extend(probs[:, 1].cpu().numpy())
    
    val_metrics = calculate_all_metrics(val_labels, val_preds, val_probs)
    print_metrics(val_metrics, "验证集 ")
    models_results['RandomForest']['cv'].append(val_metrics)
    
    # 测试
    test_preds, test_labels, test_probs = [], [], []
    with torch.no_grad():
        for x, y in tqdm(test_loader, desc="随机森林测试"):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(y.cpu().numpy())
            test_probs.extend(probs[:, 1].cpu().numpy())
    
    test_metrics = calculate_all_metrics(test_labels, test_preds, test_probs)
    print_metrics(test_metrics, "测试集 ")
    models_results['RandomForest']['test'].append(test_metrics)
    
    if test_metrics['ACC'] > best_test_acc['RandomForest']:
        best_test_acc['RandomForest'] = test_metrics['ACC']
        best_test_fold['RandomForest'] = fold
        best_test_metrics['RandomForest'] = test_metrics
    
    # ========== 梯度提升 ==========
    print(f"\n--- 梯度提升 Fold {fold} ---")
    
    model = GradientBoostingGPU().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    
    # 训练
    model.train()
    for epoch in range(35):
        for x, y in tqdm(train_loader, desc=f"梯度提升训练 Epoch {epoch+1}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()
    
    # 验证
    model.eval()
    val_preds, val_labels, val_probs = [], [], []
    with torch.no_grad():
        for x, y in tqdm(val_loader, desc="梯度提升验证"):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(y.cpu().numpy())
            val_probs.extend(probs[:, 1].cpu().numpy())
    
    val_metrics = calculate_all_metrics(val_labels, val_preds, val_probs)
    print_metrics(val_metrics, "验证集 ")
    models_results['GradientBoosting']['cv'].append(val_metrics)
    
    # 测试
    test_preds, test_labels, test_probs = [], [], []
    with torch.no_grad():
        for x, y in tqdm(test_loader, desc="梯度提升测试"):
            x, y = x.to(device), y.to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(y.cpu().numpy())
            test_probs.extend(probs[:, 1].cpu().numpy())
    
    test_metrics = calculate_all_metrics(test_labels, test_preds, test_probs)
    print_metrics(test_metrics, "测试集 ")
    models_results['GradientBoosting']['test'].append(test_metrics)
    
    if test_metrics['ACC'] > best_test_acc['GradientBoosting']:
        best_test_acc['GradientBoosting'] = test_metrics['ACC']
        best_test_fold['GradientBoosting'] = fold
        best_test_metrics['GradientBoosting'] = test_metrics


========== Fold 1/10 ==========
训练集样本数: 4134, 验证集样本数: 460

--- 逻辑回归 Fold 1 ---


逻辑回归验证: 100%|██████████| 4/4 [00:00<00:00, 16.20it/s]


验证集 ACC: 0.8761, PRE: 0.8817, REC: 0.9543, F1: 0.9165, MCC: 0.6855
验证集 SN: 0.9543, SP: 0.6818, AUC: 0.9330, AUPRC: 0.9702


逻辑回归测试: 100%|██████████| 9/9 [00:00<00:00, 18.09it/s]


测试集 ACC: 0.8816, PRE: 0.8875, REC: 0.9548, F1: 0.9199, MCC: 0.7012
测试集 SN: 0.9548, SP: 0.7009, AUC: 0.9287, AUPRC: 0.9633

--- SVM Fold 1 ---


...

测试集 ACC: 0.9164, PRE: 0.9402, REC: 0.9425, F1: 0.9414, MCC: 0.7959
测试集 SN: 0.9425, SP: 0.8520, AUC: 0.9644, AUPRC: 0.9816

--- 梯度提升 Fold 10 ---


梯度提升验证: 100%|██████████| 4/4 [00:00<00:00, 14.49it/s]


验证集 ACC: 0.8954, PRE: 0.9515, REC: 0.8991, F1: 0.9245, MCC: 0.7580
验证集 SN: 0.8991, SP: 0.8864, AUC: 0.9433, AUPRC: 0.9627


梯度提升测试: 100%|██████████| 9/9 [00:00<00:00, 25.80it/s]

测试集 ACC: 0.8982, PRE: 0.9488, REC: 0.9059, F1: 0.9268, MCC: 0.7619
测试集 SN: 0.9059, SP: 0.8792, AUC: 0.9553, AUPRC: 0.9763


In [16]:
# 汇总结果
print(f"\n{'='*80}")
print("========== 十折交叉验证结果汇总 ==========")
print(f"{'='*80}")

for model_name in models_results.keys():
    print(f"\n--- {model_name} ---")
    
    # 验证集结果 - 显示所有9个指标
    cv_metrics = models_results[model_name]['cv']
    cv_acc = [m['ACC'] for m in cv_metrics]
    cv_pre = [m['PRE'] for m in cv_metrics]
    cv_rec = [m['REC'] for m in cv_metrics]
    cv_f1 = [m['F1'] for m in cv_metrics]
    cv_mcc = [m['MCC'] for m in cv_metrics]
    cv_sn = [m['SN'] for m in cv_metrics]
    cv_sp = [m['SP'] for m in cv_metrics]
    cv_auc = [m['AUC'] for m in cv_metrics]
    cv_auprc = [m['AUPRC'] for m in cv_metrics]
    
    print(f"验证集 - ACC: {np.mean(cv_acc):.4f} ± {np.std(cv_acc):.4f}")
    print(f"验证集 - PRE: {np.mean(cv_pre):.4f} ± {np.std(cv_pre):.4f}")
    print(f"验证集 - REC: {np.mean(cv_rec):.4f} ± {np.std(cv_rec):.4f}")
    print(f"验证集 - F1:  {np.mean(cv_f1):.4f} ± {np.std(cv_f1):.4f}")
    print(f"验证集 - MCC: {np.mean(cv_mcc):.4f} ± {np.std(cv_mcc):.4f}")
    print(f"验证集 - SN:  {np.mean(cv_sn):.4f} ± {np.std(cv_sn):.4f}")
    print(f"验证集 - SP:  {np.mean(cv_sp):.4f} ± {np.std(cv_sp):.4f}")
    print(f"验证集 - AUC: {np.mean(cv_auc):.4f} ± {np.std(cv_auc):.4f}")
    print(f"验证集 - AUPRC: {np.mean(cv_auprc):.4f} ± {np.std(cv_auprc):.4f}")
    
    # 最佳测试结果 - 显示所有9个指标
    best_metrics = best_test_metrics[model_name]
    print(f"最佳测试集 (Fold {best_test_fold[model_name]}) - ACC: {best_metrics['ACC']:.4f}")
    print(f"最佳测试集 - PRE: {best_metrics['PRE']:.4f}, REC: {best_metrics['REC']:.4f}")
    print(f"最佳测试集 - F1: {best_metrics['F1']:.4f}, MCC: {best_metrics['MCC']:.4f}")
    print(f"最佳测试集 - SN: {best_metrics['SN']:.4f}, SP: {best_metrics['SP']:.4f}")
    print(f"最佳测试集 - AUC: {best_metrics['AUC']:.4f}, AUPRC: {best_metrics['AUPRC']:.4f}")


========== 十折交叉验证结果汇总 ==========

--- LogisticRegression ---
验证集 - ACC: 0.8790 ± 0.0060
验证集 - PRE: 0.8824 ± 0.0074
验证集 - REC: 0.9578 ± 0.0094
验证集 - F1:  0.9185 ± 0.0040
验证集 - MCC: 0.6942 ± 0.0158
验证集 - SN:  0.9578 ± 0.0094
验证集 - SP:  0.6841 ± 0.0245
验证集 - AUC: 0.9244 ± 0.0124
验证集 - AUPRC: 0.9561 ± 0.0133
最佳测试集 (Fold 4) - ACC: 0.8851
最佳测试集 - PRE: 0.8898, REC: 0.9572
最佳测试集 - F1: 0.9223, MCC: 0.7103
最佳测试集 - SN: 0.9572, SP: 0.7069
最佳测试集 - AUC: 0.9292, AUPRC: 0.9637

--- SVM ---
验证集 - ACC: 0.9038 ± 0.0148
验证集 - PRE: 0.9277 ± 0.0178
验证集 - REC: 0.9386 ± 0.0182
验证集 - F1:  0.9329 ± 0.0103
验证集 - MCC: 0.7645 ± 0.0371
验证集 - SN:  0.9386 ± 0.0182
验证集 - SP:  0.8179 ± 0.0485
验证集 - AUC: 0.9500 ± 0.0119
验证集 - AUPRC: 0.9743 ± 0.0066
最佳测试集 (Fold 1) - ACC: 0.9199
最佳测试集 - PRE: 0.9504, REC: 0.9364
最佳测试集 - F1: 0.9433, MCC: 0.8071
最佳测试集 - SN: 0.9364, SP: 0.8792
最佳测试集 - AUC: 0.9598, AUPRC: 0.9822

--- RandomForest ---
验证集 - ACC: 0.9103 ± 0.0097
验证集 - PRE: 0.9346 ± 0.0169
验证集 - REC: 0.9404 ± 0.0144
验证集 - F1:  0

In [7]:

# 直接用全部训练集训练和测试四个 GPU 版模型

# Cell 1: 训练和测试 LogisticRegressionGPU
train_loader = DataLoader(full_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LogisticRegressionGPU().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(20):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} finished.")

model.eval()
test_preds, test_labels, test_probs = [], [], []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(y.cpu().numpy())
        test_probs.extend(probs[:, 1].cpu().numpy())
metrics = calculate_all_metrics(test_labels, test_preds, test_probs)
print_metrics(metrics, "测试集 ")

# Cell 2: 训练和测试 SVMGPU
model = SVMGPU().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(30):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} finished.")

model.eval()
test_preds, test_labels, test_probs = [], [], []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(y.cpu().numpy())
        test_probs.extend(probs[:, 1].cpu().numpy())
metrics = calculate_all_metrics(test_labels, test_preds, test_probs)
print_metrics(metrics, "测试集 ")

# Cell 3: 训练和测试 RandomForestGPU
model = RandomForestGPU().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(25):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} finished.")

model.eval()
test_preds, test_labels, test_probs = [], [], []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(y.cpu().numpy())
        test_probs.extend(probs[:, 1].cpu().numpy())
metrics = calculate_all_metrics(test_labels, test_preds, test_probs)
print_metrics(metrics, "测试集 ")

# Cell 4: 训练和测试 GradientBoostingGPU
model = GradientBoostingGPU().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(35):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} finished.")

model.eval()
test_preds, test_labels, test_probs = [], [], []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        probs = torch.softmax(logits, dim=1)
        preds = torch.argmax(logits, dim=1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(y.cpu().numpy())
        test_probs.extend(probs[:, 1].cpu().numpy())
metrics = calculate_all_metrics(test_labels, test_preds, test_probs)
print_metrics(metrics, "测试集 ")


Epoch 1 finished.
Epoch 2 finished.
Epoch 3 finished.
Epoch 4 finished.
Epoch 5 finished.
Epoch 6 finished.
Epoch 7 finished.
Epoch 8 finished.
Epoch 9 finished.
Epoch 10 finished.
Epoch 11 finished.
Epoch 12 finished.
Epoch 13 finished.
Epoch 14 finished.
Epoch 15 finished.
Epoch 16 finished.
Epoch 17 finished.
Epoch 18 finished.
Epoch 19 finished.
Epoch 20 finished.
测试集 ACC: 0.8877, PRE: 0.8937, REC: 0.9560, F1: 0.9238, MCC: 0.7173
测试集 SN: 0.9560, SP: 0.7190, AUC: 0.9306, AUPRC: 0.9645
Epoch 1 finished.
Epoch 2 finished.
Epoch 3 finished.
Epoch 4 finished.
Epoch 5 finished.
Epoch 6 finished.
Epoch 7 finished.
Epoch 8 finished.
Epoch 9 finished.
Epoch 10 finished.
Epoch 11 finished.
Epoch 12 finished.
Epoch 13 finished.
Epoch 14 finished.
Epoch 15 finished.
Epoch 16 finished.
Epoch 17 finished.
Epoch 18 finished.
Epoch 19 finished.
Epoch 20 finished.
Epoch 21 finished.
Epoch 22 finished.
Epoch 23 finished.
Epoch 24 finished.
Epoch 25 finished.
Epoch 26 finished.
Epoch 27 finished.
Epo